In [1]:
import re
import random
import itertools
import pandas as pd
import numpy as np
import seaborn as sns
from tqdm import tqdm
from gensim.models import phrases
import  matplotlib.pyplot as plt
from gensim.models import Word2Vec
from gensim.models.phrases import  Phraser,Phrases
from nltk.corpus import stopwords

In [2]:
def read_files():#Read All Text Files
    
    file=open("Siddhartha.txt",encoding="utf8")
    corpus1=file.read()
    file=open("The-Adventures-of-Sherlock-Holmes.txt",encoding="utf8")
    corpus2=file.read()
    file=open("If-Not-For-The-Knight.txt",encoding="utf8")
    corpus3=file.read()
    data=corpus1+corpus2+corpus3


    return data   

In [3]:
stopwords=stopwords.words("english")
stopwords

['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 "you're",
 "you've",
 "you'll",
 "you'd",
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his',
 'himself',
 'she',
 "she's",
 'her',
 'hers',
 'herself',
 'it',
 "it's",
 'its',
 'itself',
 'they',
 'them',
 'their',
 'theirs',
 'themselves',
 'what',
 'which',
 'who',
 'whom',
 'this',
 'that',
 "that'll",
 'these',
 'those',
 'am',
 'is',
 'are',
 'was',
 'were',
 'be',
 'been',
 'being',
 'have',
 'has',
 'had',
 'having',
 'do',
 'does',
 'did',
 'doing',
 'a',
 'an',
 'the',
 'and',
 'but',
 'if',
 'or',
 'because',
 'as',
 'until',
 'while',
 'of',
 'at',
 'by',
 'for',
 'with',
 'about',
 'against',
 'between',
 'into',
 'through',
 'during',
 'before',
 'after',
 'above',
 'below',
 'to',
 'from',
 'up',
 'down',
 'in',
 'out',
 'on',
 'off',
 'over',
 'under',
 'again',
 'further',
 'then',
 'once',
 'here',
 'there',
 'when',
 'where',
 'why',
 'how',
 'all',
 'any',
 'both',
 'each

In [4]:
#Remove all unnasasary words
def clean_data(sentence):
    sentence=re.sub(r'[^A-Za-z0-9\s.]',r'',str(sentence).lower())
    sentence=re.sub(r'\n',r'',sentence)
    sentence=" ".join([word for word in sentence.split() if word not in stopwords])
    return sentence

In [5]:
#Shuffle data
def suffle_data(sentence):
    suffled=list(sentence)
    random.shuffle(suffled)
    return suffled

In [6]:
data=read_files() #Save all in one variable

In [7]:
data=data.splitlines() #Split all lines
data=list(filter(None,data)) #Remove blank data
data=pd.DataFrame(data) #Convert in dataFrame

In [8]:
data.head(10)

,0
0,"The Project Gutenberg EBook of Siddhartha, by ..."
1,Copyright laws are changing all over the world...
2,This header should be the first thing seen whe...
3,"Please read the “legal small print,” and other..."
4,**Welcome To The World of Free Plain Vanilla E...
5,**eBooks Readable By Both Humans and By Comput...
6,*****These eBooks Were Prepared By Thousands o...
7,Title: Siddhartha
8,Author: Herman Hesse
9,"Release Date: February, 2001 [EBook #2500] [Mo..."


In [9]:
data[0]=data[0].map(lambda x:clean_data(x))  #Apply to clean data
temp_corp=data[0].map(lambda x:x.split('.')) #Apply to split the sentence in word

In [10]:
corpus=[] #append all word to corpus
for i in tqdm(range(len(temp_corp))):
    for line in temp_corp[i][:-1]:
        word=[x for x in line.split()]
        corpus.append(word)

100%|██████████| 10377/10377 [00:00<00:00, 85918.41it/s]


In [11]:
number_of_sentence=len(corpus)
number_word=0
for i in corpus:
    number_word+=len(i)
print("number_of sentence",number_of_sentence)
print("number of word",number_word)

#To Detect common pharase and combine it in to single word
phrases = Phrases(corpus, min_count=25, threshold=50)
bigram = Phraser(phrases)

for index,sentence in enumerate(corpus):
    corpus[index]=bigram[sentence]

number_of sentence 13985
number of word 92452


In [12]:
model=Word2Vec(sentences=corpus,sg=1,window=2, min_count=2, workers=4,iter=100,sample=0.01)

In [13]:
#Train the Model
for i in tqdm(range(5)):
    model.train(sentences=suffle_data(corpus),epochs=50,total_examples=model.corpus_count)

100%|██████████| 5/5 [01:03<00:00, 12.61s/it]


In [14]:
#Save the model for future use 
model.save("w2v_model")
model=Word2Vec.load("w2v_model")

In [15]:
#Test the model
model.wv.most_similar("boy")

[('timid', 0.42580246925354004),
 ('helping', 0.42322707176208496),
 ('minor', 0.4203213155269623),
 ('irritation', 0.41295334696769714),
 ('lifted', 0.40578845143318176),
 ('firmly', 0.405303031206131),
 ('blessing', 0.39499086141586304),
 ('weapons', 0.39014092087745667),
 ('reluctant', 0.3876863121986389),
 ('ha', 0.38426440954208374)]

In [16]:
model.wv.most_similar("buddha")

[('jetavana', 0.5554880499839783),
 ('perfected', 0.5323502421379089),
 ('sakyamuni', 0.5032818913459778),
 ('novices', 0.500219464302063),
 ('everywhere', 0.48540061712265015),
 ('dicegambler', 0.4842549264431),
 ('gotama', 0.48248612880706787),
 ('shines', 0.4748327136039734),
 ('follower', 0.46940064430236816),
 ('obsessed', 0.4689693748950958)]